In [1]:
!pip install stanza
import stanza
stanza.download('en')
nlp = stanza.Pipeline('en')
from collections import defaultdict

     |████████████████████████████████| 235kB 3.4MB/s 


2020-08-19 14:31:09 INFO: Downloading default packages for language: en (English)...
2020-08-19 14:31:53 INFO: Finished downloading models and saved to /root/stanza_resources.
2020-08-19 14:31:53 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| sentiment | sstplus   |
| ner       | ontonotes |

2020-08-19 14:31:53 INFO: Use device: gpu
2020-08-19 14:31:53 INFO: Loading: tokenize
2020-08-19 14:32:01 INFO: Loading: pos
2020-08-19 14:32:02 INFO: Loading: lemma
2020-08-19 14:32:02 INFO: Loading: depparse
2020-08-19 14:32:03 INFO: Loading: sentiment
2020-08-19 14:32:05 INFO: Loading: ner
2020-08-19 14:32:05 INFO: Done loading processors!


In [2]:
from collections import OrderedDict
main=OrderedDict()

In [3]:
"""
Log into your google drive account and make sure all the tourist review datasets are in the my drive folder 
"""
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [4]:
import os
os.chdir('My Drive')
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
"""
Again login to drive account to load the datasheet
"""
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [8]:
"""
All the datasets are imported here Change the name of the dataset accordingly
"""
worksheet = gc.open('MachuPicchu_Reviews_SIGIR').sheet1
rows = worksheet.get_all_values()
import pandas as pd
df = pd.DataFrame(rows)
df.columns = df.iloc[0]
df = df.iloc[1:]

worksheet = gc.open('GreatWall_Reviews_SIGIR').sheet1
rows = worksheet.get_all_values()
df1 = pd.DataFrame(rows)
df1.columns = df1.iloc[0]
df1 = df1.iloc[1:]
df=df.append(df1, ignore_index = True) 

worksheet = gc.open(' Itza_Reviews_SIGIR').sheet1
rows = worksheet.get_all_values()
df1 = pd.DataFrame(rows)
df1.columns = df1.iloc[0]
df1 = df1.iloc[1:]
df=df.append(df1, ignore_index = True) 

worksheet = gc.open('Christ_Reviews').sheet1
rows = worksheet.get_all_values()
df1 = pd.DataFrame(rows)
df1.columns = df1.iloc[0]
df1 = df1.iloc[1:]
df=df.append(df1, ignore_index = True) 

worksheet = gc.open('Petra_Reviews_SIGIR').sheet1
rows = worksheet.get_all_values()
df1 = pd.DataFrame(rows)
df1.columns = df1.iloc[0]
df1 = df1.iloc[1:]
df=df.append(df1, ignore_index = True) 

worksheet = gc.open('Taj_Reviews_SIGIR').sheet1
rows = worksheet.get_all_values()
df1 = pd.DataFrame(rows)
df1.columns = df1.iloc[0]
df1 = df1.iloc[1:]
df=df.append(df1, ignore_index = True) 

worksheet = gc.open('Colosseum_Reviews_SIGIR').sheet1
rows = worksheet.get_all_values()
df1 = pd.DataFrame(rows)
df1.columns = df1.iloc[0]
df1 = df1.iloc[1:]
df=df.append(df1, ignore_index = True) 

df.size

1975155

In [12]:
pos=2
col_result=df.loc[:,'review_body']
asp_result=df.loc[:,'Aspects']
senti_result=df.loc[:,'Sentiments']

In [13]:
from collections import defaultdict
dataset=[]
for ii in range(1,len(col_result)):
  text=col_result[ii]
  text=text.strip()
  text=sent_tokenize(text) 
  asp=asp_result[ii]
  asp=asp.strip()
  asp=asp.split(";")
  senti=senti_result[ii]
  senti=senti.strip()
  senti=senti.split(';')
  if(len(text)==len(asp) & len(text)==len(senti) & len(asp)==len(senti)):
    for i in range(len(text)):
      doc=nlp(text[i])
      dicts = doc.to_dict()
      dd = defaultdict(list)
      for l in dicts:
        for d in l:
          for key, value in d.items():
              dd[key].append(value)
      dd=dict(dd)
      main={}
      main["token"]=dd['text']
      main["pos"]=dd['xpos']
      main["head"]=dd['head']
      main["deprel"]=dd["deprel"]
      main["aspects"]=[]
      A=asp[i]
      X=['']
      A=A.split(",")
      S=senti[i]
      S=S.split(",")
      if len(A)!=len(S):
        continue;
      for j in range(len(A)):
        temp={}
        temp["term"]=A[j]
        if(S[j]=="0"):
          temp["polarity"]="neutral"
        if(S[j]=="1"):
          temp["polarity"]="positive"
        if(S[j]=="-1"):
          temp["polarity"]="negative"
        if(A[j]!='$'):
          keyword=A[j]
          sentence=text[i]
          words = sentence.split(' ')
          if keyword in words:
              pos = words.index(keyword)
          temp["from"]=pos
          temp["to"]=pos+1
          main["aspects"].append(temp)
      if(asp[i]!='$'):
        if(A!=X):
          dataset.append(main)
          print(A)
          print(ii)

['trek', 'mountains']
97
['view']
97
['guide', 'pools']
97
['elevation']
97
['experience']
399
['hiked']
399
['park']
399
['"iPhone camera"']
399
['place', 'mountain']
556
['views']
556
['weathers']
556
['site']
740
['history']
740
['weather']
740
['crowded']
740
['rain']
740
['"train ride"']
740
['crowds']
740
['waits', 'weather']
740
['tour']
761
['tour']
761
['"low season"', 'visitors']
761
['rain']
761
['"mosquito repellent"']
761
['bus ride']
761
['place']
783
['place']
783
['ruins', 'pictures']
783
['travel']
783
['peru']
783
['"rainy season"']
785
['hike']
785
['ruins']
785
['bus', 'hiking']
785
['walk', 'bus']
819
['walking']
819
['people']
819
['view']
819
['"Inka bridge"']
819
['guide']
823
['"New Seven World Wonders"']
823
['"all other seven"']
823
['time']
848
['"grounds and views"']
848
['"bus ride"']
848
['"private tour guide"']
848
['he(guide)']
848
['photos']
848
['time']
848
['place']
1018
['history', 'mountains']
1018
['walls']
1018
['stairs']
1018
['visiting']
1018
[

In [16]:
traindata=[]
for i in range(1200):
  traindata.append(dataset[i])
testdata=[]
for i in range(1200,len(dataset)):
  testdata.append(dataset[i])

import json
with open('traindata.json', 'w') as f:
    json.dump(traindata, f)
with open('testdata.json', 'w') as f:
    json.dump(testdata, f)
""" To download the json file"""
from google.colab import files
files.download('traindata.json')

from google.colab import files
files.download('testdata.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>